<a href="https://colab.research.google.com/github/Meyyaou/pythondaily/blob/main/STEP4-Webscrape/webscrape_project_(minione).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#import all tools
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml
import re

In [38]:
url="https://www.amazon.fr/s?rh=n%3A302004%2Cp_72%3A4-&content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&ref=Oct_d_otopr_S"

In [ ]:
page=requests.get(url)
page

In [40]:
content=page.text
#content

In [41]:
soup=BeautifulSoup(content, 'lxml')

In [42]:
#to get readable html post parsing
soup.prettify()

'<html>\n <head>\n  <title>\n   Toutes nos excuses\n  </title>\n  <style type="text/css">\n   <!--\n.serif { font-family: times,serif; font-size: small; }\n.sans { font-family: verdana,arial,helvetica,sans-serif; font-size: small; }\n.small { font-family: verdana,arial,helvetica,sans-serif; font-size: x-small; }\n.h1 { font-family: verdana,arial,helvetica,sans-serif; color: #CC6600; font-size: small; }\n.h3color { font-family: verdana,arial,helvetica,sans-serif; color: #CC6600; font-size: x-small; }\n.tiny { font-family: verdana,arial,helvetica,sans-serif; font-size: xx-small; }\n.listprice { font-family: arial,verdana,helvetica,sans-serif; text-decoration: line-through; font-size: x-small; }\n.price { font-family: verdana,arial,helvetica,sans-serif; color: #990000; font-size: x-small; }\n-->\n  </style>\n </head>\n <body alink="#FF9933" bgcolor="#FFFFFF" link="#003399" text="#000000" vlink="#996633">\n  <!--\n        To discuss automated access to Amazon data please contact api-servic

In [26]:
#tronquer what we dont need!
for header in soup.find_all('header'):
  header.decompose()
soup.find('div', class_='navLeftFooter').decompose()
soup.find('div', class_='rhf-frame').decompose()
soup.find('div', class_='copilot-secure-display').decompose()

[]

In [43]:
#get the title
titles=[]
pattern =re.compile(r'\d+')

for title in soup.find_all('span', class_='a-text-normal'):
  if not pattern.match(title.string):
    titles.append(title.string)
    title
titles

[]

In [46]:
#get auteur
auteurs=[]

for auteur in soup.find_all('span', class_='a-size-base'):
  auteurs.append(auteur.string)
auteurs


0

In [10]:
#get date de sortie

In [11]:
#get price

In [12]:
#get price livraison

In [13]:
#get price kindle

In [14]:
#get ordre dans le top!

In [15]:
#DATA ANALYSIS

In [50]:
#email stuff doesnt work for gmail
import smtplib
from email.message import EmailMessage

In [62]:
senderEmail='sender@gmail.com'
recieverEmail = 'reciever@gmail.com'
password = input(str("please enter your pswd: "))
msg= "hey that's a quick test lol !"
server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.ehlo()
server.login(senderEmail, password)
print('login successful!')
server.sendmail(senderEmail, recieverEmail, msg)
print("email has been sent to reciever")
server.close()

KeyboardInterrupt: Interrupted by user